In [2]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [2]:
# model_track_info_df = pd.read_excel('../data/Enformer/41592_2021_1252_MOESM3_ESM.xlsx', sheet_name='Supplementary Table 2')
# model_track_info_df

In [3]:
model_track_info_df = pd.read_csv("../data/Enformer/model_track_info.tsv", sep="\t")
model_track_info_df

,index,genome,identifier,file,clip,scale,sum_stat,description,assay_type,target,assay_subtype,DNASE: cell-type agnostic,CAGE: cell-type agnostic,CAGE: IRF6,CAGE: TERT-GBM,CAGE: ZFAND3,"CAGE: HNF4A,MSMB,TERT-HEK293T,MYCrs6983267","CAGE: GP1BB,HBB,HBG1,PKLR","CAGE: F9,LDLR,SORT1",DNASE: IRF4,DNASE: IRF6,DNASE: ZFAND3,"DNASE: HNF4A,MSMB,TERT-HEK293T,MYCrs6983267","DNASE: GP1BB,HBB,HBG1,PKLR","DNASE: F9,LDLR,SORT1"
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...,DNASE,DNase/cerebellum male adult (27 years) and mal...,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...,DNASE,DNase/frontal cortex male adult (27 years) and...,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion,DNASE,DNase/chorion,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,3,0,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...,DNASE,DNase/Ishikawa treated with 0.02% dimethyl sul...,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,4,0,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348,DNASE,DNase/GM03348,DNase,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5308,5308,0,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R,CAGE,CAGE/epithelioid sarcoma cell line:HS-ES-2R,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False
5309,5309,0,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...,CAGE,CAGE/squamous cell lung carcinoma cell line:RE...,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False
5310,5310,0,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS,CAGE,CAGE/gastric cancer cell line:GSS,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False
5311,5311,0,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727,CAGE,CAGE/carcinoid cell line:NCI-H727,CAGE,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [4]:
# filtered_df = model_track_info_df[
#     model_track_info_df['target'].str.contains('HepG2', na=False) &
#     model_track_info_df['target'].str.contains('CAGE', na=False)
# ]
# filtered_df

In [5]:
def parse_target(t):
    assay, cell_type = t.split('/')
    if assay == 'CAGE':
        if ':' in cell_type:
            cell_desc, cell_type = cell_type.split(':')
    elif assay.startswith('ChIP-Histone'):
        assay = assay.replace('ChIP-Histone:', '')
    elif assay.startswith('ChIP-TF'):
        assay = assay.replace('ChIP-TF:', '')

    if 'ENCODE' in cell_type:
        cell_type = cell_type.replace('ENCODE, biol_', '')
    cell_type = cell_type.strip(' ')
    return assay, cell_type

In [6]:
model_track_info_df[['assay', 'cell_type']] = model_track_info_df['target'].apply(parse_target).apply(pd.Series)
model_track_info_df = model_track_info_df[['assay', 'cell_type']]
model_track_info_df = model_track_info_df.reset_index()
model_track_info_df

,index,assay,cell_type
0,0,DNase,cerebellum male adult (27 years) and male adul...
1,1,DNase,frontal cortex male adult (27 years) and male ...
2,2,DNase,chorion
3,3,DNase,Ishikawa treated with 0.02% dimethyl sulfoxide...
4,4,DNase,GM03348
...,...,...,...
5308,5308,CAGE,HS-ES-2R
5309,5309,CAGE,RERF-LC-AI
5310,5310,CAGE,GSS
5311,5311,CAGE,NCI-H727


In [7]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']

for cell_type in cell_types:
    mask = model_track_info_df['cell_type'].str.contains(cell_type, na=False)
    filtered_df = model_track_info_df[mask]
    print(filtered_df['cell_type'].value_counts())

cell_type
K562                                                                        388
K562 genetically modified using stable transfection                          54
K562 genetically modified using CRISPR                                       19
K562 treated with interferon alpha for 30 minutes                             6
K562 treated with interferon gamma for 6 hours                                5
K562 treated with interferon alpha for 6 hours                                5
K562 treated with interferon gamma for 30 minutes                             4
K562 G1 phase                                                                 1
K562 G2 phase                                                                 1
K562 treated with 1 uM vorinostat for 72 hours                                1
K562 genetically modified using stable transfection originated from K562      1
Name: count, dtype: int64
cell_type
HepG2                                      168
HepG2 genetically modified 

In [8]:
df_pivot = model_track_info_df.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)

df_pivot = df_pivot.map(lambda x: x if isinstance(x, list) else [])
df_pivot.map(len)

assay,.,3xFLAG-AHR,3xFLAG-ARID4B,3xFLAG-ATF1,3xFLAG-ATF4,3xFLAG-BCL6,3xFLAG-CEBPA,3xFLAG-CEBPG,3xFLAG-CREB1,3xFLAG-DMAP1,3xFLAG-DNMT3B,3xFLAG-DRAP1,3xFLAG-ELF3,3xFLAG-ERF,3xFLAG-ETV5,3xFLAG-FOXA3,3xFLAG-FOXP1,3xFLAG-GABPA,3xFLAG-GABPB1,3xFLAG-GATAD1,3xFLAG-GATAD2A,3xFLAG-GMEB2,3xFLAG-HBP1,3xFLAG-HHEX,3xFLAG-HLF,3xFLAG-HMG20A,3xFLAG-HMG20B,3xFLAG-HOMEZ,3xFLAG-IKZF5,3xFLAG-IRF2,3xFLAG-KAT7,3xFLAG-KAT8,3xFLAG-KDM1A,3xFLAG-KDM3A,3xFLAG-KDM6A,3xFLAG-KLF11,3xFLAG-KLF16,3xFLAG-KLF6,3xFLAG-KLF9,3xFLAG-KMT2B,3xFLAG-MBD1,3xFLAG-MIER2,3xFLAG-MIER3,3xFLAG-MIXL1,3xFLAG-MLX,3xFLAG-MXD3,3xFLAG-MXD4,3xFLAG-MYRF,3xFLAG-NFIA,3xFLAG-NFIL3,3xFLAG-NFKBIZ,3xFLAG-NFYC,3xFLAG-NR2F1,3xFLAG-NR2F6,3xFLAG-PAF1,3xFLAG-PBX2,3xFLAG-PHF5A,3xFLAG-PPARG,3xFLAG-RAD21,3xFLAG-RARA,3xFLAG-RBPJ,3xFLAG-RCOR2,3xFLAG-RERE,3xFLAG-RFX3,3xFLAG-RFXANK,3xFLAG-RXRB,3xFLAG-SAP130,3xFLAG-SLC30A9,3xFLAG-SMAD4,3xFLAG-SOX13,3xFLAG-SOX5,3xFLAG-SP5,3xFLAG-SSRP1,3xFLAG-TEAD1,3xFLAG-TEAD3,3xFLAG-TFE3,3xFLAG-TGIF2,3xFLAG-THAP11,3xFLAG-THRB,3xFLAG-ZBED5,3xFLAG-ZBTB25,3xFLAG-ZBTB26,3xFLAG-ZFP1,3xFLAG-ZFP64,3xFLAG-ZGPAT,3xFLAG-ZKSCAN8,3xFLAG-ZNF146,3xFLAG-ZNF205,3xFLAG-ZNF3,3xFLAG-ZNF48,3xFLAG-ZNF511,3xFLAG-ZNF580,3xFLAG-ZNF614,3xFLAG-ZNF639,3xFLAG-ZNF644,3xFLAG-ZNF652,3xFLAG-ZNF7,3xFLAG-ZNF792,3xFLAG-ZSCAN9,AFF1,AGO1,AGO2,ARHGAP35,ARID1B,ARID2,ARID3A,ARNT,ASH1L,ASH2L,ATAC,ATF2,ATF3,ATF4,ATF7,ATM,BACH1,BATF,BCL11A,BCL3,BCLAF1,BCOR,BHLHE40,BMAL1,BMI1,BRCA1,BRD4,BRD9,C11orf30,CAGE,CBFA2T2,CBFA2T3,CBFB,CBX1,CBX2,CBX3,CBX5,CBX8,CC2D1A,CCAR2,CDC5L,CEBPB,CEBPZ,CEBPb,CHAMP1,CHD1,CHD2,CHD4,CHD7,CLOCK,COPS2,CREB1,CREB3L1,CREBBP,CREM,CSDE1,CTBP1,CTBP2,CTCF,CUX1,DACH1,DDX20,DEAF1,DEK,DNMT1,DNase,DPF2,E2F1,E2F4,E2F6,E2F7,E2F8,E4F1,EBF1,EED,EGR1,EHMT2,ELF1,ELF4,ELK1,EP300,EP400,ESR1,ESRRA,ETS1,ETV4,ETV6,EWSR1,EZH2,EZH2phosphoT487,FIP1L1,FOS,FOSL1,FOSL2,FOXA1,FOXA2,FOXK2,FOXM1,FOXP1,FOXP2,FUS,GABPA,GABPB1,GATA1,GATA2,GATA3,GATA4,GATAD2A,GATAD2B,GMEB1,GR,GTF2F1,H2AFZ,H2AK5ac,H2AK9ac,H2BK120ac,H2BK12ac,H2BK15ac,H2BK20ac,H2BK5ac,H3F3A,H3K14ac,H3K18ac,H3K23ac,H3K23me2,H3K27Ac,H3K27ac,H3K27me3,H3K36me3,H3K4ac,H3K4me1,H3K4me2,H3K4me3,H3K56ac,H3K79me1,H3K79me2,H3K9ac,H3K9me1,H3K9me2,H3K9me3,H3T11ph,H4K12ac,H4K20me1,H4K5ac,H4K8ac,H4K91ac,HCFC1,HDAC1,HDAC2,HDAC3,HDAC6,HDGF,HES1,HLTF,HMBOX1,HNF1A,HNF4A,HNF4G,HNRNPH1,HNRNPK,HNRNPL,HNRNPLL,HNRNPUL1,HSF1,IKZF1,IKZF2,ILF3,IRF1,IRF2,IRF3,IRF4,IRF5,JUN,JUNB,JUND,KAT2A,KAT2B,KAT8,KDM1A,KDM4A,KDM4B,KDM5A,KDM5B,KHSRP,KLF16,KLF5,L3MBTL2,LARP7,LCORL,LEF1,MAFF,MAFK,MAX,MAZ,MBD2,MCM2,MCM3,MCM5,MCM7,MEF2A,MEF2B,MEF2C,MEIS2,MGA,MIER1,MITF,MLLT1,MNT,MTA1,MTA2,MTA3,MXI1,MYB,MYBL2,MYC,MYNN,NANOG,NBN,NCOA1,NCOA2,NCOA3,NCOA4,NCOA6,NCOR1,NEUROD1,NFATC1,NFATC3,NFE2,NFE2L2,NFIB,NFIC,NFRKB,NFXL1,NFYA,NFYB,NKRF,NONO,NR0B1,NR2C1,NR2C2,NR2F1,NR2F2,NR2F6,NR3C1,NRF1,NUFIP1,PAX5,PAX8,PBX3,PCBP1,PCBP2,PHB2,PHF20,PHF21A,PHF8,PKNOX1,PLRG1,PML,POLR2A,POLR2AphosphoS2,POLR2AphosphoS5,POLR2B,POLR2G,POU2F2,POU5F1,PPP1R10,PRDM10,PRPF4,PTBP1,PYGO2,RAD21,RAD51,RB1,RBBP5,RBFOX2,RBM14,RBM15,RBM17,RBM22,RBM25,RBM34,RBM39,RCOR1,RELB,REST,RFX1,RFX5,RLF,RNAPII,RNF2,RUNX1,RUNX3,RXRA,SAFB,SAFB2,SAP30,SETDB1,SFPQ,SIN3A,SIN3B,SIRT6,SIX4,SIX5,SKI,SKIL,SMAD1,SMAD2,SMAD5,SMARCA4,SMARCA5,SMARCB1,SMARCC2,SMARCE1,SMC3,SNIP1,SNRNP70,SOX13,SOX6,SP1,SPI1,SREBF1,SREBF2,SRF,SRSF1,SRSF3,SRSF4,SRSF7,SRSF9,STAT1,STAT2,STAT3,STAT5A,SUPT20H,SUPT5H,SUZ12,SYNCRIP,TAF1,TAF15,TAF7,TAF9B,TAL1,TARDBP,TBL1XR1,TBP,TBX21,TBX3,TCF12,TCF7,TCF7L2,TEAD4,TFAP4,THAP1,THRA,THRAP3,TOE1,TRIM22,TRIM24,TRIM25,TRIM28,TRIP13,U2AF1,U2AF2,UBTF,USF1,USF2,WHSC1,WRNIP1,XRCC3,XRCC5,YBX1,YBX3,YY1,ZBED1,ZBTB1,ZBTB11,ZBTB2,ZBTB33,ZBTB40,ZBTB5,ZBTB7A,ZBTB7B,ZBTB8A,ZC3H11A,ZC3H8,ZEB1,ZEB2,ZFP36,ZFP91,ZFX,ZHX1,ZHX2,ZKSCAN1,ZMIZ1,ZMYM3,ZNF143,ZNF184,ZNF207,ZNF217,ZNF24,ZNF263,ZNF274,ZNF280A,ZNF282,ZNF316,ZNF318,ZNF384,ZNF407,ZNF444,ZNF507,ZNF512B,ZNF574,ZNF579,ZNF592,ZNF622,ZNF639,ZNF687,ZNF8,ZNF830,ZSCAN29,ZZZ3,abcam,active,eGFP-ADNP,eGFP-AEBP2,eGFP-ATF1,eGFP-ATF2,eGFP-ATF3,eGFP-BACH1,eGFP-BCL11A,eGFP-BCL11B,eGFP-BCL6B,eGFP-CEBPB,eGFP-CEBPG,eGFP-CREB3,eGFP-CT

In [9]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF', 'CAGE']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549', 'SK-N-SH treated with 6 uM all-trans-retinoic acid for 48 hours', 'HepG2 genetically modified using CRISPR']

df_pivot.loc[cell_types, assays].map(len)

assay,DNase,H3K4me3,H3K27ac,CTCF,CAGE
cell_type,,,,,
K562,4,4,0,4,2
HepG2,3,3,0,1,1
SK-N-SH,0,2,2,2,0
HCT116,1,2,2,3,0
A549,1,1,0,2,1
SK-N-SH treated with 6 uM all-trans-retinoic acid for 48 hours,1,1,0,1,0
HepG2 genetically modified using CRISPR,0,0,0,0,0


In [10]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF', 'CAGE']
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']

df_track = df_pivot.loc[cell_types, assays]

df_track.loc['SK-N-SH', 'DNase'] = df_track.loc['SK-N-SH', 'DNase'] + df_pivot.loc['SK-N-SH treated with 6 uM all-trans-retinoic acid for 48 hours', 'DNase']

df_track

assay,DNase,H3K4me3,H3K27ac,CTCF,CAGE
cell_type,,,,,
K562,"[121, 122, 123, 625]","[701, 1210, 1387, 3507]",[],"[697, 1012, 1211, 1347]","[4828, 5111]"
HepG2,"[27, 91, 234]","[717, 1182, 3227]",[],[904],[5109]
SK-N-SH,[146],"[1441, 4439]","[1443, 3194]","[1371, 3110]",[]
HCT116,[88],"[1173, 2467]","[1379, 3485]","[1055, 1172, 2203]",[]
A549,[47],[1102],[],"[1084, 1242]",[4852]


In [11]:
pred_array = load_h5('outputs/Gosai_MPRA_Enformer_pred.h5')
print(pred_array.shape)

file outputs/Gosai_MPRA_Enformer_pred.h5 has keys: ['data']


(760679, 2, 5313)


In [12]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_my_processed_data_len200_norm.csv')
print(MPRA_df.shape)

(760679, 14)


In [13]:
VEF_df = pd.DataFrame()

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]
        if isinstance(indice, list) and len(indice) > 0:
            for k in indice:
                pred = pred_array[:, :, k].mean(1)
                true = MPRA_df[cell_type]
                r = pearson(pred, true)[0]
                print(cell_type, assay, k, r)
            pred = pred_array[:, :, indice].mean(2).mean(1)
            VEF_df[f'{cell_type}_{assay}'] = pred
        else:
            VEF_df[f'{cell_type}_{assay}'] = np.nan

K562 DNase 121 0.4816071363277534
K562 DNase 122 0.4644253456390126
K562 DNase 123 0.4131292168590919
K562 DNase 625 0.4816063723944516
K562 H3K4me3 701 0.5021894400924606
K562 H3K4me3 1210 0.4376592465589576
K562 H3K4me3 1387 0.4032735113679041
K562 H3K4me3 3507 0.3936855959824625
K562 CTCF 697 0.20051753499180994
K562 CTCF 1012 0.2907718944359341
K562 CTCF 1211 0.20491694559927867
K562 CTCF 1347 0.3171555814117574
K562 CAGE 4828 0.08247017994303404
K562 CAGE 5111 0.11835509544869822
HepG2 DNase 27 0.27880721326309493
HepG2 DNase 91 0.31399730755736893
HepG2 DNase 234 0.2954469807537239
HepG2 H3K4me3 717 0.40260261905845723
HepG2 H3K4me3 1182 0.3148607856280888
HepG2 H3K4me3 3227 0.2340740827940256
HepG2 CTCF 904 0.1678087544761365
HepG2 CAGE 5109 0.15406251712904728
SK-N-SH DNase 146 0.36282836567717996
SK-N-SH H3K4me3 1441 0.44859695312120584
SK-N-SH H3K4me3 4439 0.3681771060157597
SK-N-SH H3K27ac 1443 0.3477368490947369
SK-N-SH H3K27ac 3194 0.3401635739666725
SK-N-SH CTCF 1371 0.14

In [14]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

pearson_df = pd.DataFrame(dtype=float)
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        pred = VEF_df[f'{cell_type}_{assay}']
        true = MPRA_df[f'{cell_type}']
        r, _ = pearson(pred, true)
        pearson_df.loc[cell_type, assay] = r
print(pearson_df)

after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan
         DNase  H3K4me3  H3K27ac  CTCF
K562     0.487    0.451      NaN 0.244
HepG2    0.356    0.359      NaN 0.168
SK-N-SH  0.363    0.447    0.380 0.152
HCT116   0.439    0.501    0.521 0.172
A549     0.361    0.344      NaN 0.175


In [15]:
VEF_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_Enformer_VEF_raw.tsv', sep='\t', index=False)

In [4]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_my_processed_data_len200_norm.csv')
VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Enformer_VEF_raw.tsv', sep='\t')
VEF_df.describe()

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,K562_CAGE,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HepG2_CAGE,SK-N-SH_DNase,SK-N-SH_H3K4me3,SK-N-SH_H3K27ac,SK-N-SH_CTCF,SK-N-SH_CAGE,HCT116_DNase,HCT116_H3K4me3,HCT116_H3K27ac,HCT116_CTCF,HCT116_CAGE,A549_DNase,A549_H3K4me3,A549_H3K27ac,A549_CTCF,A549_CAGE
count,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,760679.000,760679.000,0.000,760679.000,760679.000,0.000,760679.000,760679.000
mean,0.088,0.763,NaN,1.125,0.160,0.112,0.684,NaN,1.150,0.472,0.127,0.979,2.326,1.124,NaN,0.147,1.160,3.005,1.160,NaN,0.113,0.532,NaN,1.006,0.062
std,0.203,0.941,NaN,1.271,0.844,0.154,0.659,NaN,0.771,1.391,0.433,1.027,5.922,1.167,NaN,0.433,2.509,7.817,1.207,NaN,0.320,1.241,NaN,0.656,0.474
min,0.002,0.061,NaN,0.108,0.002,0.005,0.052,NaN,0.108,0.000,0.006,0.081,0.032,0.125,NaN,0.006,0.099,0.126,0.127,NaN,0.003,0.031,NaN,0.067,0.001
25%,0.038,0.575,NaN,0.824,0.053,0.047,0.523,NaN,0.916,0.121,0.047,0.736,0.526,0.917,NaN,0.058,0.670,0.932,0.939,NaN,0.042,0.279,NaN,0.746,0.019
50%,0.056,0.645,NaN,0.981,0.090,0.075,0.606,NaN,1.115,0.221,0.064,0.834,0.750,1.014,NaN,0.077,0.767,1.247,1.074,NaN,0.064,0.336,NaN,0.959,0.032
75%,0.081,0.735,NaN,1.178,0.160,0.132,0.691,NaN,1.277,0.445,0.091,0.931,1.378,1.117,NaN,0.102,0.878,1.800,1.185,NaN,0.098,0.424,NaN,1.170,0.055
max,7.030,38.720,NaN,50.780,127.900,6.470,32.400,NaN,28.530,164.000,18.030,43.880,122.560,44.030,NaN,14.340,64.300,139.800,41.800,NaN,12.740,44.970,NaN,23.730,81.560


In [5]:
VEF_df = np.log1p(VEF_df)
VEF_df.to_csv('../data/Gosai_MPRA/Gosai_MPRA_Enformer_VEF_log1p.tsv', sep='\t', index=False)